# 텍스트 파일 및 csv 파일에서 문장 단위로 분해하기 (SENTENCE TOKENIZE)
https://cryptosalamander.tistory.com/140

In [1]:
import os
import re
from tqdm import tqdm
import pandas as pd
import nltk
from nltk.data import load

`nltk.sent_tokenize`에 Abbreviations 추가하기

In [2]:
tokenizer = load("tokenizers/punkt/english.pickle")

extra_abbreviations = [
    'RE','re','pat', 'no', 'nos','vol','jan','feb','mar','apr','jun',
    'jul','aug','sep','oct','nov','dec','eng','ser','ind','ed','pp',
    'e.g','al','T.E.N.S', 'E.M.S','F.E','U.H.T.S.T','degree',
    '/gm','A','B','C','D','E','F','G','H','I','J','K','L','M','N','O',
    'P','Q','R','S','T','U','V','W','X','Y','Z',
    
    'inc', 'K.L.M', 'u.s', 'U.S', 'A.C.U', 'A.S.U', 'A.T.C.T', 'A.T.M.S', 
    'A.P.T', 'A.R.C', 'A.D.R', 'B.C.C', 'B.S.A', 'C.A.O', 'C.B.A', 'C.T.A', 
    'C.C', 'C.C.A', 'P.P', 'C.P.M', 'F.B.O', 'F.O.C', 'F.O.B', 'G.P', 'G.P.U', 
    'L.C.C', 'L.C', 'L.D.C', 'M.D.C', 'N.I.L', 'O.S.I', 'O.L.T', 'P.D.U', 
    'S.B.C', 'T.V', 'G.B.L', 'V.I.C'
]
tokenizer._params.abbrev_types.update(extra_abbreviations)

In [3]:
def pre_process_doc(doc):
    doc = re.sub(r'\s+', ' ', doc)
    doc = doc.replace("“", '"').replace("”", '"')
    doc = doc.replace('‘', "'").replace('’', "'")
    sent_list = nltk.sent_tokenize(doc)
    processed_doc = '\n'.join(sent_list)
    return processed_doc 

In [4]:
def pre_process_text(text):
    text = re.sub(r'\s+', ' ', text)
    text = text.replace("“", '"').replace("”", '"')
    text = text.replace('‘', "'").replace('’', "'")
        
    return text

In [5]:
def pre_process_text2(text):
    text = text.replace('ㅇ', "\n- ")
    text = re.sub(r'\s+', ' ', text)
    text = text.replace("“", '"').replace("”", '"')
    text = text.replace('‘', "'").replace('’', "'")
        
    return text

## (1) 사고준사고보고서 (`*.txt`)

In [6]:
input_folder = 'rawdata/사고준사고보고서'  # 텍스트 파일들이 있는 폴더 경로
output_folder = 'train'  # 전처리된 파일을 저장할 폴더 경로

# 입력 폴더 내의 모든 텍스트 파일 경로 가져오기
file_paths = []
for root, dirs, files in os.walk(input_folder):
    for file in files:
        if file.endswith('.txt'):
            file_paths.append(os.path.join(root, file))

In [7]:
df = pd.DataFrame(index=range(0, len(file_paths)), columns = ['text'])

In [8]:
# 각 파일 불러와 전처리 후 저장
for idx, file_path in tqdm(enumerate(file_paths)):
    # 텍스트 파일 읽기
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()

    # 전처리 수행
    doc = '\n'.join(lines)
    processed_doc = pre_process_doc(doc)
    
    df["text"][idx] = processed_doc

59it [00:00, 133.05it/s]


In [9]:
# DataFrame을 csv 파일로 저장
df.to_csv(os.path.join(output_folder, '사고준사고보고서.csv'), index=False)

---
## (2-1) 항공안전문화지표 (`*.csv`)

In [10]:
csv_file_path = 'rawdata/항공안전문화지표/항공안전문화지표 분석 모델.csv'  # 읽어올 CSV 파일 경로
output_folder = 'train/'  # 텍스트 파일을 저장할 폴더 경로

file_name = os.path.basename(csv_file_path)  # 파일 이름 추출
output_path = os.path.join(output_folder, file_name)

# CSV 파일을 데이터프레임으로 읽기
df = pd.read_csv(csv_file_path)

In [11]:
df["text"] = df["본문"]

for i in tqdm(range(0, len(df)-1)):
    df["text"][i] = "- 보고서: "
    df["text"][i]  += re.sub(r'\s+', ' ', df["본문"][i])
        
    suffix = "- 항공안전 업무영역: "
    if pd.isna(df["항공안전 업무영역1"][i]) == False:
        suffix += df["항공안전 업무영역1"][i].replace('\n', ', ')
    if pd.isna(df["항공안전 업무영역2"][i]) == False:
        if suffix == "- 항공안전 업무영역: ":
            suffix += df["항공안전 업무영역2"][i].replace('\n', ', ')
        else:
            suffix += ", " + df["항공안전 업무영역2"][i].replace('\n', ', ')
    if suffix != "- 항공안전 업무영역: ":
        df["text"][i] += "\n" + suffix
    
    suffix = "- 원인분석 & 전조징후: "
    if pd.isna(df["원인분석 - 전조징후 1"][i]) == False:
        suffix += df["원인분석 - 전조징후 1"][i].replace('\n', ', ')
    
        if pd.isna(df["원인분석 - 전조징후 2"][i]) == False:
            suffix += ", " + df["원인분석 - 전조징후 2"][i].replace('\n', ', ')
            
            if pd.isna(df["원인분석 - 전조징후 3"][i]) == False:
                suffix += ", " + df["원인분석 - 전조징후 3"][i].replace('\n', ', ')
    if suffix != "- 원인분석 - 전조징후: ":
        df["text"][i] += "\n" + suffix
        
    suffix_list = []
    for row in ["안전문화1", "안전문화2", "안전문화3", "안전문화4", "안전문화5", "안전문화6"]:
        if pd.isna(df[row][i]) == False:
            suffix_list.append(df[row][i].replace('\n', ', '))
    suffix_list = list(set(suffix_list))
    if len(suffix_list)>0:
        suffix = "- 안전문화 : "
        suffix += ", ".join(suffix_list)
        df["text"][i] += "\n" + suffix
    
    
    
df = df[["text"]]
df.to_csv(output_path, index=False)

100% 1055/1055 [00:01<00:00, 652.95it/s]


## (2-2) 항공안전문화지표 증강 (`*.csv`)

In [12]:
csv_file_path = 'rawdata/항공안전문화지표/항공안전문화지표 분석모델_증강.csv'  # 읽어올 CSV 파일 경로
output_folder = 'train/'  # 텍스트 파일을 저장할 폴더 경로

file_name = os.path.basename(csv_file_path)  # 파일 이름 추출
output_path = os.path.join(output_folder, file_name)

# CSV 파일을 데이터프레임으로 읽기
df = pd.read_csv(csv_file_path)

In [13]:
df["text"] = df["main"]

for i in tqdm(range(len(df))):
    df["text"][i] = pre_process_doc(df["text"][i])
df = df[["text"]]

  0% 0/5902 [00:00<?, ?it/s]/tmp/ipykernel_4935/2379416453.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text"][i] = pre_process_doc(df["text"][i])
100% 5902/5902 [00:04<00:00, 1202.24it/s]


In [14]:
df.to_csv(output_path, index=False)

---
## (3) 항공용어사전

In [15]:
csv_file_path = 'rawdata/항공용어사전/국토교통부_항공용어사전_20200916.csv'  # 읽어올 CSV 파일 경로
output_folder = 'train/'  # 텍스트 파일을 저장할 폴더 경로

file_name = os.path.basename(csv_file_path)  # 파일 이름 추출
output_path = os.path.join(output_folder, file_name)

# CSV 파일을 데이터프레임으로 읽기
df = pd.read_csv(csv_file_path)

In [16]:
df

,용어,한글명,약칭,해설
0,A & B hydraulic brake system,A & B 유압 제동 장치,NaN,일부 대형 항공기의 다중디스크(multi-disk) 동력 제동 장치에 사용되는 예비...
1,A & P technician,기체 및 동력장치 정비사,NaN,FAA에서 인증한 항공기 정비 경력과 지식을 갖춘 기체 및 동력장치(airframe...
2,A-10 thunderbolt II,A-10 선더볼트 II,NaN,미공군은 베트남 전쟁 후 보다 강력한 근접지원(close support)항공기의 필...
3,A-37 dragonfly,A-37 드레곤플라이,NaN,미공군의 표준 기본 훈련기(basic trainer)로 활약하고 있으며 날개아래 파...
4,A-battery,A-배터리,NaN,일반적으로 1.5볼트(volt)에서 6.0볼트의 전압을 발생시킬 수 있는 배터리.
...,...,...,...,...
4956,zero fuel weight,"무연료 무게, 무연료 중량",NaN,"항공기가 연료, 윤활유를 탑재치 않았을 때 허용되는 중량. 이 중량을 설정한 것은 ..."
4957,zero gravity,무중력,NaN,관성력에 의해서 중력의 효과가 상실되는 상태. 무중력은 고성능 항공기에서는 포물선 ...
4958,zero lift angle of attack,무받음각,NaN,양력을 발생하지 못한다.
4959,zoom up,급상승 고도 획득 비행,NaN,전속 수평직선비행으로 기체를 가속하여 그 여세로 한번에 급상승하여 고도를 획득하는 ...


In [17]:
df["text"] = df["약칭"]

for i in tqdm(range(len(df))):
    df["text"][i] = ""
    
    if pd.isna(df["용어"][i]) == False:
        df["text"][i] += f'- 용어 : {df["용어"][i]}\n'
    if pd.isna(df["한글명"][i]) == False:
        df["text"][i] += f'- 한글명 : {df["한글명"][i]}\n'
    if pd.isna(df["약칭"][i]) == False:
        df["text"][i] += f'- 약칭 : {df["약칭"][i]}\n'
              
    df["text"][i] += "- 해설 :" + pre_process_text(df["해설"][i])  # "해설" 열의 값을 텍스트로 저장
    
df = df[["text"]]
df.to_csv(output_path, index=False)

100% 4961/4961 [00:05<00:00, 885.14it/s]


---
## (4-1) 고장보고

In [18]:
csv_file_path = 'rawdata/안전장애_고장보고_데이터/고장보고 데이터.csv'  # 읽어올 CSV 파일 경로
output_folder = 'train/'  # 텍스트 파일을 저장할 폴더 경로

file_name = os.path.basename(csv_file_path)  # 파일 이름 추출
output_path = os.path.join(output_folder, file_name)

# CSV 파일을 데이터프레임으로 읽기
df = pd.read_csv(csv_file_path)

In [19]:
df["text"] = df["접수번호"]

# 각 행을 텍스트 파일로 저장
for i in tqdm(range(len(df))):
    text = df["고장내용"][i]  # "고장내용" 열의 값을 텍스트로 저장
    df["text"][i] = pre_process_text2(text)
    
    if pd.isna(df["고장 등의 상태"][i]) == False:
        df["text"][i] += "\n" + f'- 고장 등의 상태 : {df["고장 등의 상태"][i]}' + '\n'
    if pd.isna(df["조치사항"][i]) == False:
        df["text"][i] += "\n" + f'- 조치사항 : {df["조치사항"][i]}' + '\n'
    if pd.isna(df["비행단계"][i]) == False:
        df["text"][i] += "\n" + f'- 비행단계 : {df["비행단계"][i]}'
              
    
df = df[["text"]]
df.to_csv(output_path, index=False)

100% 1771/1771 [00:01<00:00, 892.44it/s]


## (4-2) 안전장애보고

In [20]:
csv_file_path = 'rawdata/안전장애_고장보고_데이터/안전장애보고 데이터.csv'  # 읽어올 CSV 파일 경로
output_folder = 'train/'  # 텍스트 파일을 저장할 폴더 경로

file_name = os.path.basename(csv_file_path)  # 파일 이름 추출
output_path = os.path.join(output_folder, file_name)

# CSV 파일을 데이터프레임으로 읽기
df = pd.read_csv(csv_file_path)

In [21]:
enter_char = '\n'

df["text"] = df["발생장소"]

# 각 행을 텍스트 파일로 저장
for i in tqdm(range(len(df))):
    text = df["발생개요"][i]  # "발생개요" 열의 값을 텍스트로 저장
    df["text"][i] = "- 발생개요 : " + pre_process_text2(text)
    
    if pd.isna(df["발생장소"][i]) == False:
        df["text"][i] += "\n" + f'- 발생장소 : {df["발생장소"][i]}\n'
    if pd.isna(df["이벤트 항목\n(보고받은 항목)"][i]) == False:
        df["text"][i] += "\n" + f'- 이벤트 항목 (보고받은 항목) : {df[f"이벤트 항목{enter_char}(보고받은 항목)"][i]}\n'
    if pd.isna(df["이벤트 항목(대)\n-분석결과"][i]) == False:
        df["text"][i] += "\n" + f'- 이벤트 항목(대)-분석결과 : {df[f"이벤트 항목(대){enter_char}-분석결과"][i]}\n'
    if pd.isna(df["이벤트 항목(중)\n-분석결과"][i]) == False:
        df["text"][i] += "\n" + f'- 이벤트 항목(중)-분석결과 : {df[f"이벤트 항목(중){enter_char}-분석결과"][i]}\n'
    if pd.isna(df["이벤트 항목(소)\n-분석결과"][i]) == False:
        df["text"][i] += "\n" + f'- 이벤트 항목(소)-분석결과 : {df[f"이벤트 항목(소){enter_char}-분석결과"][i]}\n'
    if pd.isna(df["요인구분(대)"][i]) == False:
        df["text"][i] += "\n" + f'- 요인구분(대) : {df["요인구분(대)"][i]}\n'
    if pd.isna(df["요인구분(중)"][i]) == False:
        df["text"][i] += "\n" + f'- 요인구분(중) : {df["요인구분(중)"][i]}\n'
    if pd.isna(df["기타특이사항"][i]) == False:
        df["text"][i] += "\n" + f'- 기타특이사항 : {df["기타특이사항"][i]}\n'
    if pd.isna(df["발생원인 분석\n(감독관님 작성)"][i]) == False:
        df["text"][i] += "\n" + f'- 발생원인 분석 : {df[f"발생원인 분석{enter_char}(감독관님 작성)"][i]}\n'
    if pd.isna(df["항공사\n귀책여부"][i]) == False:
        df["text"][i] += "\n" + f'- 항공사 귀책여부 : {df[f"항공사{enter_char}귀책여부"][i]}\n'
    if pd.isna(df["안전지표\n반영여부"][i]) == False:
        df["text"][i] += "\n" + f'- 안전지표 반영여부 : {df[f"안전지표{enter_char}반영여부"][i]}\n'
    if pd.isna(df["항공사 조치"][i]) == False:
        text = df["항공사 조치"][i]  # "항공사 조치" 열의 값을 텍스트로 저장
        df["text"][i] += "\n" + "- 항공사 조치 :" + pre_process_text2(text)
    
df = df[["text"]]
df.to_csv(output_path, index=False)

100% 684/684 [00:01<00:00, 415.36it/s]


## (5) GYRO

In [22]:
csv_file_path = 'rawdata/GYRO/GYRO_trainset.csv'  # 읽어올 CSV 파일 경로
output_folder = 'train/'  # 텍스트 파일을 저장할 폴더 경로

file_name = os.path.basename(csv_file_path)  # 파일 이름 추출
output_path = os.path.join(output_folder, file_name)

# CSV 파일을 데이터프레임으로 읽기
df = pd.read_csv(csv_file_path)

In [23]:
# 각 행을 텍스트 파일로 저장

df["text"] = df["본문"]

for i in tqdm(range(len(df))):
    text = ""
    
    if pd.isna(df["사고명"][i]) == False:
        text += f'- 사고명 : {df["사고명"][i]}\n'
    
    text += f'- 본문 :\n{pre_process_doc(df["text"][i])}\n' # "text" 열의 값을 텍스트로 저장
    
    
    text += f'- 원인 키워드 : {df["원인 키워드"][i]}\n' # "text" 열의 값을 텍스트로 저장
    
    if pd.isna(df["원인"][i]) == False:
        text += f'- 원인 : {pre_process_doc(df["원인"][i])}\n'
    if pd.isna(df["발생지표"][i]) == False:
        text += f'- 발생지표 : {pre_process_doc(df["발생지표"][i])}\n'
    if pd.isna(df["요약"][i]) == False:
        text += f'- 요약 : {pre_process_doc(df["요약"][i])}\n'
    if pd.isna(df["재발방지대책"][i]) == False:
        text += f'- 재발방지대책 : {df["재발방지대책"][i]}\n'
    if pd.isna(df["조치결과"][i]) == False:
        text += f'- 조치결과 : {df["조치결과"][i]}\n'
    if pd.isna(df["DATA"][i]) == False:
        text += f'- DATA : {df["DATA"][i]}\n'
    if pd.isna(df["비고"][i]) == False:
        text += f'- 비고 : {df["비고"][i]}\n'
       
    df["text"][i] = text
    
df = df[["text"]]
df.to_csv(output_path, index=False)

  0% 0/874 [00:00<?, ?it/s]/tmp/ipykernel_4935/2970371766.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text"][i] = text
100% 874/874 [00:00<00:00, 1165.46it/s]


## (6) 항공위키

In [24]:
csv_file_path = 'rawdata/항공위키/airwiki_data.csv'  # 읽어올 CSV 파일 경로
output_folder = 'train/'  # 텍스트 파일을 저장할 폴더 경로

file_name = os.path.basename(csv_file_path)  # 파일 이름 추출
output_path = os.path.join(output_folder, file_name)

# CSV 파일을 데이터프레임으로 읽기
df = pd.read_csv(csv_file_path)

In [25]:
# 각 행을 텍스트 파일로 저장
df["text"] = df["문서 내용"]

for i in tqdm(range(len(df))):
    text = df["text"][i]  # "text" 열의 값을 텍스트로 저장
    df["text"][i] = pre_process_text(text)
    
df = df[["text"]]
df.to_csv(output_path, index=False)

  0% 0/4314 [00:00<?, ?it/s]/tmp/ipykernel_4935/2037974171.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text"][i] = pre_process_text(text)
100% 4314/4314 [00:01<00:00, 2365.42it/s]
